<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/semaforos_do_recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='40%' aling='center' src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/semaforos_do_recife/logo_recife.jpg?raw=true" />

A **Autarquia de Trânsito e Transporte Urbano do Recife (CTTU)** é uma sociedade de economia mista integrante da Administração Indireta Municipal e é responsável pelo monitoramento, controle e fiscalização do trânsito e do transporte na capital. Compete à CTTU também a regulamentação do Sistema Municipal de Transporte Público de Passageiros, que é explorado sob a forma de permissão ou autorização nos modais de táxi, transporte complementar (linhas interbairros e alimentadoras) e transporte escolar, ações de estudos e projetos de melhoria da mobilidade urbana e sua efetiva implantação e de educação para o trânsito.

*Fonte:* https://bit.ly/3svji3h

##Sobre os dados

Relação de todos os semáforos georreferenciados, contendo informações referentes ao modo de funcionamento, e tipo de semáforo.
  * Dados coletados do http://dados.recife.pe.gov.br/
    * Dataset: https://bit.ly/2XGmA8u
    * Dicionário: https://bit.ly/3kcfBLQ
    * Mais: https://bit.ly/3y20D02
  
  * Lista das variáveis:
    * _id
    * semaforo
    * localizacao1
    * localizacao2
    * bairro
    * latitude
    * longitude
    * tipo

##Obtenção e transformação dos dados

**Carrega e verifica dataset**

In [38]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

In [39]:
# bibliotecas
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
import json
import random

# mapa
#!pip install folium
import folium
from folium import plugins

In [40]:
PATH_DATA = 'http://dados.recife.pe.gov.br/dataset/ab87d3e2-ea78-4f27-95fd-d7ef6f3b9ee5/resource/ab6343e9-c3f2-4d62-9554-5778f9f33738/download/semaforos_bairro.csv'
df = pd.read_csv(PATH_DATA, error_bad_lines=False, sep=';')
df.head()

,semafororo,localizacao1,localizacao2,bairro,latitude,longitude,tipo,funcionamento
0,1,Av. Boa Viagem,Em frente ao parque Dona Lindú,Boa Viagem,-8.142323,-34.903798,E/Git,Ocasional/Ped.
1,2,Av. Conselheiro Aguiar,Rua Thomé Gibson,Pina,-8.099068,-34.885883,E/Git,Veicular
2,3,Av. Boa Viagem,Rua França Pereira,Boa Viagem,-8.106806,-34.887726,E/Git,Ocasional/Ped.
3,4,Av. Boa Viagem,Rua Barão de Souza Leão,Boa Viagem,-8.132230,-34.899990,E/Git,Ocasional/Ped.
4,5,Av. Conselheiro Aguiar,Rua Artur Muniz,Boa Viagem,-8.104310,-34.888228,E/Git,Veicular


In [41]:
# verificando dados ausentes
df.isnull().sum()

semafororo       0
localizacao1     0
localizacao2     0
bairro           0
latitude         0
longitude        0
tipo             0
funcionamento    0
dtype: int64

In [42]:
# tipo de dados
df.dtypes

semafororo         int64
localizacao1      object
localizacao2      object
bairro            object
latitude         float64
longitude        float64
tipo              object
funcionamento     object
dtype: object

In [43]:
# estatísticas
df.describe()

,semafororo,latitude,longitude
count,670.000000,670.000000,670.000000
mean,336.714925,-8.069820,-34.905234
std,194.546134,0.034403,0.019903
min,1.000000,-8.151128,-34.968542
25%,168.250000,-8.097905,-34.916216
50%,336.500000,-8.061868,-34.903201
75%,504.750000,-8.043458,-34.889949
max,673.000000,-8.004059,-34.870413


**Transformação**

Por enquanto, não foi preciso formatar os dados. Nosso dataset provavelmente passou por um tratamento antes de ser compartilhado.

##Visualização

In [44]:
# tabela de cores
px.colors.qualitative.swatches()

In [45]:
coluna = 'bairro'
grafico = px.histogram(data_frame=df, x=coluna, template='plotly_white', color_discrete_sequence=px.colors.qualitative.T10)
grafico.show()

In [46]:
coluna = "funcionamento"
grafico = px.histogram(data_frame=df, 
                       x=coluna,
                       color="funcionamento"
                       )
grafico.show()

In [47]:
df['bairro'].value_counts()[:5]

Boa Viagem       97
Imbiribeira      43
Santo Amaro      40
Santo Antônio    27
São José         26
Name: bairro, dtype: int64

Top 5 dos bairros que possuem mais semáforos.

**Mapa de calor**

In [48]:
# criando coordenadas
coordenadas=[]
for lat, lng in zip(df.latitude, df.longitude):
  coordenadas.append([lat, lng])

# criando mapa
mapa = folium.Map(
    location=[-8.0517123,-34.9281543], 
    zoom_start=12, 
    tiles='OpenStreetMap' # tipo do mapa: Stamen Toner
    )

# exibi mapa
mapa.add_child(plugins.HeatMap(coordenadas))

**Mapa com marcador**

In [49]:
# cria mapa
mapa2 = folium.Map(
    location=[-8.0704276,-34.894891], 
    zoom_start=12, 
    tiles='OpenStreetMap' # tipo do mapa: Stamen Toner
)


# cria pontos
for i, row in df.iterrows():
    latitude = df.at[i, 'latitude']
    longitude = df.at[i, 'longitude']
    bairro = df.at[i, 'bairro']

    folium.Marker(
        location=[latitude,longitude], 
        popup=bairro,
        icon = folium.Icon(color="blue")  
    ).add_to(mapa2)

#mapa2.save('index.html')
mapa2

# tutorial: https://www.youtube.com/watch?v=9biKWoGK3j0&ab_channel=Code%26Dogs

##Conclusão

Em poucos gráficos, podemos ver a quantidade de semáforos espalhados pela cidade. Muitos deles podem ser julgados como desnecessários, reforçando a tese sobre da Industria das Multas, mas, também, em muitos casos realmente é de serventia para sociedade, fazendo a gestão do tráfego. 

O bairro de Boa Viagem é onde se concentra o maior número deles, totalizando 97. Logo em seguida está a Imbiribeira, com 43 e fechando o top 3, vem Santo Amaro com 40.